## EDA 과제 1_문제1

- 서울시 스타벅스 매장의 이름 / 주소 / 구 이름을 pandas data frame으로 정리

In [245]:
# importing selenium, webdriver, BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import googlemaps
from haversine import haversine

In [180]:
# opening a tab with selenium (스타벅스 매장 찾기)

URL = "https://www.starbucks.co.kr/store/store_map.do?disp=locale"
driver = webdriver.Chrome("../driver/chromedriver.exe")
driver.get(URL) # 스타벅스 매장 찾기 url

In [181]:
# clicking 매장찾기, 지역검색, 서울, 전체 / html 가져오기

find_store = driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > header.find_store_header > h2')
find_store.click()
time.sleep(1.5)

local_search = driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search')
local_search.click()
time.sleep(1.5)

Seoul_select = driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a')
Seoul_select.click()
time.sleep(1.5)

whole_select = driver.find_element(By.CSS_SELECTOR, '#mCSB_2_container > ul > li:nth-child(1) > a')
whole_select.click()
time.sleep(3)

html = driver.page_source

In [182]:
# 스타벅스 매장 이름, 주소, 해당 구 리스트 만들기

soup = BeautifulSoup(html, "html.parser")
SB_name = soup.select('.quickSearchResultBoxSidoGugun > .quickResultLstCon')
SB_address = soup.select('.quickSearchResultBoxSidoGugun > .quickResultLstCon > .result_details')

SB_name_list = [SB_name[n]['data-name'] for n in range(len(SB_name))]
SB_gu_list = [SB_address[n].text.split()[1] for n in range(len(SB_address))]
SB_address_list = [SB_address[n].text[:-9] for n in range(len(SB_address))]
SB_lat = [SB_name[n]['data-lat'] for n in range(len(SB_name))]
SB_long = [SB_name[n]['data-long'] for n in range(len(SB_name))]

In [183]:
SB_data = pd.DataFrame({
    "매장 이름" : SB_name_list,
    "구" : SB_gu_list,
    "주소" : SB_address_list,
    "lat" : SB_lat,
    "lng" : SB_long
})

In [237]:
SB_data['lat'] = SB_data['lat'].astype(float)
SB_data['lng'] = SB_data['lng'].astype(float)
SB_data

,매장 이름,구,주소,lat,lng
0,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,강남구,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606
3,국기원사거리,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩R,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...
598,사가정역,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
599,상봉역,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.596890,127.086470
600,묵동,중랑구,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
601,양원역,중랑구,서울특별시 중랑구 양원역로10길 3 (망우동),37.606654,127.106360


In [185]:
# 스타벅스 결과 엑셀 저장
SB_data.to_excel("../data/SB_result.xlsx")

## EDA 과제 1_문제2

- 서울시 이디야 매장의 이름 / 주소 / 구 이름을 pandas data frame으로 정리

In [186]:
# 이디야 커피 매장 찾기 (1) opening new tab with ediya coffee, 다루는 탭 변경

driver.execute_script('window.open("https://www.ediya.com/contents/find_store.html#c")')
driver.switch_to.window(driver.window_handles[1]) # 없으면 맨 처음 스타벅스 탭에서 작업을 진행하게 됨.

In [187]:
# 이디야 커피 매장 찾기 (2) 주소 클릭과 구 입력하기
# + 지점 이름과 행정구역, 주소 저장하기

ed_click = driver.find_element(By.CSS_SELECTOR, "#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a").click()
ed_keyword = driver.find_element(By.CSS_SELECTOR, "#keyword")

ed_seoul_gu = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구',
               '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구',
               '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', 
               '송파구', '강동구']

ed_name_list = []
ed_gu_list = []
ed_address_list = []
ed_lat = []
ed_lng = []

for n in ed_seoul_gu:
    ed_keyword.send_keys("서울 " + n)
    driver.find_element(By.CSS_SELECTOR, "#keyword_div > form > button").click()
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    for x in range(len(soup.select('#placesList > .item'))):
        ed_name_list.append(str(soup.select('#placesList > .item')[x].select('dt')[0]).replace("<dt>", "").replace("</dt>", "")) # 지점 이름
        ed_address_list.append(str(soup.select('#placesList > .item')[x].select('dd')[0]).replace("<dd>", "").replace("</dd>", "")) # 주소
        ed_gu_list.append(str(soup.select('#placesList > .item')[x].select('dd')[0]).replace("<dd>", "").replace("</dd>", "").split()[1]) # 행정구역
    ed_keyword.clear()
    time.sleep(1.5)


In [224]:
# 이디야 커피숍 관련 데이터 프레임 만들기 & 위도 경도 데이터 넣을 준비

ED_data = pd.DataFrame({
    "매장 이름" : ed_name_list,
    "구" : ed_gu_list,
    "주소" : ed_address_list,
})
ED_data['lat'] = np.nan
ED_data['lng'] = np.nan
ED_data.tail()

,매장 이름,구,주소,lat,lng
632,위례아이파크점,송파구,"서울 송파구 위례광장로 136 (장지동, 위례아이파크)",NaN,NaN
633,위례포레샤인점,송파구,"서울 송파구 위례순환로 477 (장지동, 위례포레샤인23단지)",NaN,NaN
634,잠실본동점,송파구,"서울 송파구 백제고분로 172 (잠실동, 제이케이빌딩)",NaN,NaN
635,잠실역점,송파구,"서울 송파구 송파대로 567 (잠실동, 잠실주공아파트)",NaN,NaN
636,종합운동장역점,송파구,"서울 송파구 올림픽로 76 (잠실동, J타워)",NaN,NaN


In [225]:
# 매장 이름을 넣으면 안 잡히는 경우가 있음 (이디야 커피 상호가 매우 다양함)
# 그로 인해 위/경도를 잡으려면 주소 사용해야 함
# 그럼에도 불구하고 10군데 이디야 커피는 주소가 잡히지 않아서 위 경도 구할 수 없음

ED_data_tmp = ED_data.set_index("주소")
ED_data_tmp

,매장 이름,구,lat,lng
주소,,,,
서울 종로구 경희궁1길 16 (신문로2가),경희궁점,종로구,NaN,NaN
서울 종로구 세종대로23길 15 (도렴동),광화문점,종로구,NaN,NaN
"서울 종로구 율곡로 6 (중학동, 트윈 트리 빌딩)",광화문트윈트리타워점,종로구,NaN,NaN
서울 종로구 낙원동,낙원동점,종로구,NaN,NaN
서울 종로구 대학로10길 17 (동숭동),대학로점,종로구,NaN,NaN
...,...,...,...,...
"서울 송파구 위례광장로 136 (장지동, 위례아이파크)",위례아이파크점,송파구,NaN,NaN
"서울 송파구 위례순환로 477 (장지동, 위례포레샤인23단지)",위례포레샤인점,송파구,NaN,NaN
"서울 송파구 백제고분로 172 (잠실동, 제이케이빌딩)",잠실본동점,송파구,NaN,NaN


In [226]:
# 이디야 커피숍 위도 경도 알아내기

for idx, rows in ED_data_tmp.iterrows():
    ed_ID = idx
    tmp = gmaps.geocode(ed_ID, language="ko")
    
    if tmp == []:
        continue

    if len(tmp) == 2:
        tmp[1].get("formatted_address")
        tmp_gu = tmp[1].get("formatted_address")
        
        lat = tmp[1].get("geometry")["location"]["lat"]
        lng = tmp[1].get("geometry")["location"]["lng"]
        
        ED_data_tmp.loc[idx, "lat"] = lat
        ED_data_tmp.loc[idx, "lng"] = lng
                
        continue
    
    tmp[0].get("formatted_address")
        
    lat = tmp[0].get("geometry")["location"]["lat"]
    lng = tmp[0].get("geometry")["location"]["lng"]
    
    ED_data_tmp.loc[idx, "lat"] = lat
    ED_data_tmp.loc[idx, "lng"] = lng

In [230]:
# 데이터 재 정리 (스타벅스 정보와 같은 순서로)

ED_data = ED_data_tmp.reset_index()
ED_data = ED_data[['매장 이름', '구', '주소', 'lat', 'lng']]
ED_data

,매장 이름,구,주소,lat,lng
0,경희궁점,종로구,서울 종로구 경희궁1길 16 (신문로2가),37.571667,126.970702
1,광화문점,종로구,서울 종로구 세종대로23길 15 (도렴동),37.571503,126.975661
2,광화문트윈트리타워점,종로구,"서울 종로구 율곡로 6 (중학동, 트윈 트리 빌딩)",37.575313,126.979677
3,낙원동점,종로구,서울 종로구 낙원동,37.572572,126.988623
4,대학로점,종로구,서울 종로구 대학로10길 17 (동숭동),37.582086,127.003180
...,...,...,...,...,...
632,위례아이파크점,송파구,"서울 송파구 위례광장로 136 (장지동, 위례아이파크)",37.476296,127.142131
633,위례포레샤인점,송파구,"서울 송파구 위례순환로 477 (장지동, 위례포레샤인23단지)",37.476281,127.135655
634,잠실본동점,송파구,"서울 송파구 백제고분로 172 (잠실동, 제이케이빌딩)",37.506016,127.082600
635,잠실역점,송파구,"서울 송파구 송파대로 567 (잠실동, 잠실주공아파트)",NaN,NaN


In [231]:
# 이디야 결과 엑셀 저장
ED_data.to_excel("../data/ED_result.xlsx")

## EDA 과제 1_문제3

- 이디야 커피가 스타벅스 커피 매장 근처에 있는지 분석
- 선택한 방법: 각 매장의 위도 경도를 최대한 가져와 스타벅스-이디야의 직선 거리를 비교
  - 거리 제한의 기준은 근접 출점 제한에 대한 기사글 참고
  - 거리 제한 안에 들어가는 점포를 카운트하여 데이터 프레임에 추가

<br/>

- 편의점의 근접 출점 제한 거리 = 100m (refer: https://www.huffingtonpost.kr/news/articleView.html?idxno=77852)
- 명랑핫도그 근접 출점 제한 거리 = 500m (refer: http://www.bizhankook.com/bk/article/20075)
- 메가커피, 컴포즈 커피 출점 제한 거리 = 250m (refer: https://news.mt.co.kr/mtview.php?no=2023050122524477275)

In [255]:
# 가장 수가 많다고 할 수 있는 체인인 편의점과, 동종 업계의 커피점을 인용
# 100m 내 근접 점포 갯수와 250m 내 근접 점포 갯수를 추가할 예정

SB_data['ED_100m'] = np.nan     # 해당 스타벅스 매장 100m 이내 이디야 커피 점포 갯수
SB_data['ED_250m'] = np.nan     # 해당 스타벅스 매장 250m 이내 이디야 커피 점포 갯수
SB_data['ED_tot'] = np.nan      # 100m, 250m 내 매장 합계
SB_data

,매장 이름,구,주소,lat,lng,ED_100m,ED_250m,ED_tot
0,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069,NaN,NaN,NaN
1,논현역사거리,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223,NaN,NaN,NaN
2,신사역성일빌딩,강남구,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606,NaN,NaN,NaN
3,국기원사거리,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495,NaN,NaN,NaN
4,대치재경빌딩R,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
598,사가정역,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966,NaN,NaN,NaN
599,상봉역,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.596890,127.086470,NaN,NaN,NaN
600,묵동,중랑구,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633,NaN,NaN,NaN
601,양원역,중랑구,서울특별시 중랑구 양원역로10길 3 (망우동),37.606654,127.106360,NaN,NaN,NaN


In [297]:
# 거리가 가까워서 중복으로 카운트 되는 이디야 커피점 확인

ED_data['SB_100m'] = np.nan
ED_data['SB_250m'] = np.nan
ED_data['SB_tot'] = np.nan
ED_data

,매장 이름,구,주소,lat,lng,SB_100m,SB_250m,SB_tot
0,경희궁점,종로구,서울 종로구 경희궁1길 16 (신문로2가),37.571667,126.970702,NaN,NaN,NaN
1,광화문점,종로구,서울 종로구 세종대로23길 15 (도렴동),37.571503,126.975661,NaN,NaN,NaN
2,광화문트윈트리타워점,종로구,"서울 종로구 율곡로 6 (중학동, 트윈 트리 빌딩)",37.575313,126.979677,NaN,NaN,NaN
3,낙원동점,종로구,서울 종로구 낙원동,37.572572,126.988623,NaN,NaN,NaN
4,대학로점,종로구,서울 종로구 대학로10길 17 (동숭동),37.582086,127.003180,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
632,위례아이파크점,송파구,"서울 송파구 위례광장로 136 (장지동, 위례아이파크)",37.476296,127.142131,NaN,NaN,NaN
633,위례포레샤인점,송파구,"서울 송파구 위례순환로 477 (장지동, 위례포레샤인23단지)",37.476281,127.135655,NaN,NaN,NaN
634,잠실본동점,송파구,"서울 송파구 백제고분로 172 (잠실동, 제이케이빌딩)",37.506016,127.082600,NaN,NaN,NaN
635,잠실역점,송파구,"서울 송파구 송파대로 567 (잠실동, 잠실주공아파트)",NaN,NaN,NaN,NaN,NaN


In [256]:
# 위/경도 거리 비교
# haversine 모듈 사용

for idx1, rows1 in SB_data.iterrows():
    SB_compare = (SB_data['lat'][idx1], SB_data['lng'][idx1])
    ED_100 = 0
    ED_250 = 0
    ED_tot = 0

    for idx2, rows2 in ED_data.iterrows():
        ED_compare = (ED_data['lat'][idx2], ED_data['lng'][idx2])
        distance = haversine(SB_compare, ED_compare, unit = 'm')
        if distance <= 100:
            ED_100 += 1
        elif distance > 100 and distance <= 250:
            ED_250 += 1
    
    ED_tot = ED_100 + ED_250
    
    SB_data["ED_100m"][idx1] = ED_100
    SB_data["ED_250m"][idx1] = ED_250
    SB_data["ED_tot"][idx1] = ED_tot

C:\Users\User\AppData\Local\Temp\ipykernel_12932\3626634527.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SB_data["ED_100m"][idx1] = ED_100
C:\Users\User\AppData\Local\Temp\ipykernel_12932\3626634527.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SB_data["ED_250m"][idx1] = ED_250
C:\Users\User\AppData\Local\Temp\ipykernel_12932\3626634527.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SB_data["ED_tot"][idx1] = ED_tot

In [323]:
# 결과 확인

SB_data.head()

,매장 이름,구,주소,lat,lng,ED_100m,ED_250m,ED_tot
0,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069,1.0,0.0,1.0
1,논현역사거리,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223,0.0,1.0,1.0
2,신사역성일빌딩,강남구,서울특별시 강남구 강남대로 584 (논현동),37.513931,127.020606,0.0,0.0,0.0
3,국기원사거리,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495,0.0,0.0,0.0
4,대치재경빌딩R,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583,0.0,0.0,0.0


In [315]:
# 100m 근접 점포 내림차순으로 대략 확인

SB_data.sort_values('ED_100m', ascending=False).head()

,매장 이름,구,주소,lat,lng,ED_100m,ED_250m,ED_tot
124,사당,관악구,서울특별시 관악구 남부순환로 2082-25,37.476010,126.980997,2.0,0.0,2.0
0,역삼아레나빌딩,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069,1.0,0.0,1.0
535,구로디지털로,구로구,"서울특별시 구로구 디지털로33길 11, 에이스테크노8차 1층 (구로동)",37.485911,126.895587,1.0,0.0,1.0
112,가양역사거리,강서구,"서울특별시 강서구 화곡로 416 (등촌동) 가양더스카이벨리5차 지식산업센터 101,...",37.560070,126.855372,1.0,0.0,1.0
502,명동역,중구,"서울특별시 중구 퇴계로 101, 건물 전체 (충무로 1가)",37.561020,126.983120,1.0,0.0,1.0


In [316]:
# 250m 근접 점포 대략 확인

SB_data.sort_values('ED_250m', ascending=False).head()

,매장 이름,구,주소,lat,lng,ED_100m,ED_250m,ED_tot
43,학동역,강남구,서울특별시 강남구 논현로 704 (논현동),37.514649,127.030730,0.0,5.0,5.0
71,논현힐탑,강남구,서울특별시 강남구 논현로 648 (논현동),37.511558,127.032262,0.0,4.0,4.0
450,종로3가,종로구,"서울특별시 종로구 종로 113-1, 1-4층 (종로3가)",37.570553,126.990206,0.0,3.0,3.0
70,선릉로,강남구,"서울특별시 강남구 선릉로93길 22, 대흥빌딩 1층 (역삼동)",37.505053,127.046577,0.0,3.0,3.0
225,합정푸르지오,마포구,"서울특별시 마포구 월드컵로3길 14 (합정동, 마포 한강 2차 푸르지오)",37.551098,126.912307,0.0,3.0,3.0


In [327]:
# 100m 이내 1개 이상의 이디야 존재 >> 총 102 점포

SB_data[SB_data['ED_100m'] != 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 596
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   매장 이름    102 non-null    object 
 1   구        102 non-null    object 
 2   주소       102 non-null    object 
 3   lat      102 non-null    float64
 4   lng      102 non-null    float64
 5   ED_100m  102 non-null    float64
 6   ED_250m  102 non-null    float64
 7   ED_tot   102 non-null    float64
dtypes: float64(5), object(3)
memory usage: 7.2+ KB


In [326]:
# 100m 초과, 250m 이내 1개 이상의 이디야 존재 >> 총 266 점포

SB_data[SB_data['ED_250m'] != 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 1 to 602
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   매장 이름    266 non-null    object 
 1   구        266 non-null    object 
 2   주소       266 non-null    object 
 3   lat      266 non-null    float64
 4   lng      266 non-null    float64
 5   ED_100m  266 non-null    float64
 6   ED_250m  266 non-null    float64
 7   ED_tot   266 non-null    float64
dtypes: float64(5), object(3)
memory usage: 18.7+ KB


In [325]:
# 250m 이내 1개 이상의 이디야 존재 >> 총 336 점포

SB_data[SB_data['ED_tot'] != 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 0 to 602
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   매장 이름    336 non-null    object 
 1   구        336 non-null    object 
 2   주소       336 non-null    object 
 3   lat      336 non-null    float64
 4   lng      336 non-null    float64
 5   ED_100m  336 non-null    float64
 6   ED_250m  336 non-null    float64
 7   ED_tot   336 non-null    float64
dtypes: float64(5), object(3)
memory usage: 23.6+ KB


In [299]:
# 중복으로 스타벅스 근접 범위 안에 들어가는 이디야가 있는지 매장 이름으로 확인

for idx1, rows1 in ED_data.iterrows():
    ED_compare = (ED_data['lat'][idx1], ED_data['lng'][idx1])
    SB_100 = []
    SB_250 = []
    SB_tot = []

    for idx2, rows2 in SB_data.iterrows():
        SB_compare = (SB_data['lat'][idx2], SB_data['lng'][idx2])
        distance = haversine(SB_compare, ED_compare, unit = 'm')
        if distance <= 100:
            SB_100.append(str(SB_data['매장 이름'][idx2]))
        elif distance > 100 and distance <= 250:
            SB_250.append(str(SB_data['매장 이름'][idx2]))
    
    SB_tot = SB_100 + SB_250
    
    if SB_tot == []:
        continue

    ED_data["SB_100m"][idx1] = str(SB_100)
    ED_data["SB_250m"][idx1] = str(SB_250)
    ED_data["SB_tot"][idx1] = str(SB_tot)

C:\Users\User\AppData\Local\Temp\ipykernel_12932\1005001093.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ED_data["SB_100m"][idx1] = str(SB_100)
C:\Users\User\AppData\Local\Temp\ipykernel_12932\1005001093.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ED_data["SB_250m"][idx1] = str(SB_250)
C:\Users\User\AppData\Local\Temp\ipykernel_12932\1005001093.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ED_data["SB_tot"][idx1

In [320]:
ED_data.head()

,매장 이름,구,주소,lat,lng,SB_100m,SB_250m,SB_tot
0,경희궁점,종로구,서울 종로구 경희궁1길 16 (신문로2가),37.571667,126.970702,[],['경희궁로'],['경희궁로']
1,광화문점,종로구,서울 종로구 세종대로23길 15 (도렴동),37.571503,126.975661,['광화문'],"['새문안로', '광화문교보문고']","['광화문', '새문안로', '광화문교보문고']"
2,광화문트윈트리타워점,종로구,"서울 종로구 율곡로 6 (중학동, 트윈 트리 빌딩)",37.575313,126.979677,['경복궁사거리'],"['연합뉴스', '이마빌딩']","['경복궁사거리', '연합뉴스', '이마빌딩']"
3,낙원동점,종로구,서울 종로구 낙원동,37.572572,126.988623,[],['인사'],['인사']
4,대학로점,종로구,서울 종로구 대학로10길 17 (동숭동),37.582086,127.003180,"['혜화역', '마로니에공원']",['동숭로아트'],"['혜화역', '마로니에공원', '동숭로아트']"


In [314]:
ED_data_check = ED_data.dropna(subset=['SB_tot'], how='any', axis=0)
ED_data_check

,매장 이름,구,주소,lat,lng,SB_100m,SB_250m,SB_tot
0,경희궁점,종로구,서울 종로구 경희궁1길 16 (신문로2가),37.571667,126.970702,[],['경희궁로'],['경희궁로']
1,광화문점,종로구,서울 종로구 세종대로23길 15 (도렴동),37.571503,126.975661,['광화문'],"['새문안로', '광화문교보문고']","['광화문', '새문안로', '광화문교보문고']"
2,광화문트윈트리타워점,종로구,"서울 종로구 율곡로 6 (중학동, 트윈 트리 빌딩)",37.575313,126.979677,['경복궁사거리'],"['연합뉴스', '이마빌딩']","['경복궁사거리', '연합뉴스', '이마빌딩']"
3,낙원동점,종로구,서울 종로구 낙원동,37.572572,126.988623,[],['인사'],['인사']
4,대학로점,종로구,서울 종로구 대학로10길 17 (동숭동),37.582086,127.003180,"['혜화역', '마로니에공원']",['동숭로아트'],"['혜화역', '마로니에공원', '동숭로아트']"
...,...,...,...,...,...,...,...,...
620,송파문정점,송파구,서울 송파구 송이로 159 (가락동),37.492957,127.125353,[],['문정로데오'],['문정로데오']
621,송파올림픽훼미리타운점,송파구,"서울 송파구 중대로 24 (문정동, 올림픽훼밀리타운 아카데미 상가)",37.489602,127.118791,[],['문정법원로R'],['문정법원로R']
624,송파파인타운점,송파구,"서울 송파구 충민로2길 28 (장지동, 대진플라자2)",37.477243,127.127404,[],['송파NC'],['송파NC']
626,송파하비오점,송파구,"서울 송파구 송파대로 111 (문정동, 파크하비오)",37.481656,127.123452,['문정하비오'],['송파아이파크'],"['문정하비오', '송파아이파크']"


### 정보 요약 및 결론, 보완점

- 데이터 분석 결과 요약

  - 서울시 내 스타벅스 총 603개 점포 존재

  - 서울시 내 이디야 커피 총 637개 점포 존재

  - 이디야 커피 267개의 점포(서울 내 점포 수의 약 42%)가 스타벅스 336개 점포의 근접 범위 내에 있음 
    (서울 내 점포 수의 약 56%) * 근접의 기준: 메가커피, 컴포즈 커피 근접 출점 제한 거리 250m

  - 전체 점포 수의 절반 정도의 이디야 커피와 스타벅스가 250m 이내에서 상권을 공유하고 있는 것을 알 수 있음

<br/>

- 보완점

  - 해당 지점에 먼저 입점한 점포가 어느 브랜드인지까지 따져야 후발주자가 전략적으로 입점했다고 말할 수 있겠음.
    (이디야 커피가 먼저 해당 지점에 입점하였다면 스타벅스가 판매 전략을 위해 근처에 들어섰다고 할 수 있음)

  - 또한, 다른 커피 브랜드 간의 상권 공유 퍼센티지가 얼마나 되는지 비교군이 필요함
    - 다양한 커피 브랜드들 간의 근접 정도를 확인 했을 때 모두 40~60%가 250m 이내 (단순한 임시 조건) 에 있다면, 이디야나 스타벅스 모두 전략적으로 근처에 입점한 것이 아니라 일반적인 커피 브랜드의 입점 모습이라고 이야기 할 수 있음.
    - 혹은 근처 상권에 대한 경제적 이점으로 인해 입점 했을 가능성이 있음. (역세권, 주거지역, 유동인구수 등에 따른 커피 브랜드 입점)

<br/>

- 결론
  
  - 근거리에 있는 점포의 갯수만 확인했을 때는 이디야 커피는 확실히 스타벅스 커피 근처에 위치하고 있다고 말할 수 있음

  - 추가적으로 한 브랜드가 타 브랜드에 대해 의도적인 근접 출점을 하였다 하더라도, 근접 출점에 대한 제제나 제약이 각 브랜드 별로 회사 내규처럼 존재하고 법적으로 처벌할 수단이 따로 없음. 그리고 소비자 입장에서는 경쟁으로 인한 품질 상승을 기대할 수 있기 때문에 판매자 입장에서 윤리적으로 비난할 수 있으나 추후로도 적극적인 제제나 법률 제정은 생기지 않을 것으로 사료됨.

